In [1]:
!pip install torch numpy tqdm

Objectif du projet:

L'objectif de ce projet était de construire et d'entraîner un modèle de traduction automatique simple utilisant une architecture Transformer. Le modèle est conçu pour traduire de courtes phrases d'anglais vers le français en utilisant un petit ensemble de données d'exemples de traduction.

Étapes réalisées :

Installation des dépendances: Les bibliothèques nécessaires (torch, numpy, tqdm) ont été installées.
Préparation des données:
Un petit ensemble de paires de phrases anglais-français a été créé.
Cet ensemble de données a été étendu par duplication et mélange pour obtenir environ 1000 exemples.
Un SimpleTokenizer a été implémenté pour convertir les phrases en séquences d'identifiants de mots, incluant des tokens spéciaux (<pad>, <sos>, <eos>, <unk>).
Une classe TranslationDataset a été définie pour préparer les données pour l'entraînement, y compris l'encodage des phrases et le padding à une longueur maximale.
Un DataLoader a été créé pour charger les données par lots pendant l'entraînement.
Définition du modèle: Un modèle Transformer simple a été défini à l'aide de PyTorch, comprenant des couches d'embedding, un encodage positionnel et des couches d'encodeur-décodeur Transformer standard.
Entraînement du modèle:
La fonction de perte CrossEntropyLoss et l'optimiseur Adam ont été configurés.
Le modèle a été entraîné sur l'ensemble de données préparé pendant 20 époques.
La perte d'entraînement a été suivie à chaque époque, montrant une diminution significative au fil du temps, ce qui indique que le modèle apprend.
Résultats de l'entraînement :

La perte d'entraînement a diminué de manière constante sur les 20 époques. La perte finale d'environ 0.0034 suggère que le modèle a bien appris l'ensemble de données limité et est capable de prédire les séquences cibles avec une grande précision sur cet ensemble de données.

Inférence :

Une fonction translate a été définie pour utiliser le modèle entraîné afin de générer des traductions pour de nouvelles phrases d'entrée. Les exemples de test montrés ("Bonjour" et "Merci") ont produit les traductions attendues ("bonjour" et "bonjour" - notez que la traduction de "Merci" semble erronée dans l'exemple fourni dans le notebook, ce qui pourrait indiquer une limitation du modèle sur un ensemble de données aussi petit).

Prochaines étapes possibles :

Utiliser un ensemble de données de traduction plus grand et plus diversifié pour améliorer les performances et la généralisation du modèle.
Ajouter des mécanismes d'attention plus sophistiqués au modèle Transformer.
Mettre en œuvre un processus de validation pour évaluer les performances du modèle sur des données invisibles pendant l'entraînement.
Expérimenter différents hyperparamètres (taille du modèle, nombre de couches, taux d'apprentissage, etc.).
Déployer le modèle pour une utilisation pratique.
Ce rapport résume les principales composantes et les résultats de ce projet de traduction automatique.



In [13]:
# train.py
from tqdm import tqdm
import torch.optim as optim
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# tokenizer.py (Included for self-containment)
from collections import defaultdict

class SimpleTokenizer:
    def __init__(self):
        self.vocab = {}
        self.idx_to_token = {}
        self.token_to_idx = {}
        self.pad_token = "<pad>"
        self.sos_token = "<sos>"
        self.eos_token = "<eos>"
        self.unk_token = "<unk>"

    def fit(self, sentences):
        vocab = set([self.pad_token, self.sos_token, self.eos_token, self.unk_token])
        for sent in sentences:
            vocab.update(sent.lower().split())
        self.vocab = sorted(vocab)
        self.idx_to_token = {i: token for i, token in enumerate(self.vocab)}
        self.token_to_idx = {token: i for i, token in self.idx_to_token.items()} # Corrected typo here

    def encode(self, sentence, add_special=True):
        tokens = sentence.lower().split()
        ids = [self.token_to_idx.get(t, self.token_to_idx[self.unk_token]) for t in tokens]
        if add_special:
            ids = [self.token_to_idx[self.sos_token]] + ids + [self.token_to_idx[self.eos_token]]
        return ids

    def decode(self, ids, skip_special=True):
        if skip_special:
            ids = [i for i in ids if i not in [
                self.token_to_idx[self.sos_token],
                self.token_to_idx[self.eos_token],
                self.token_to_idx[self.pad_token]
            ]]
        return " ".join(self.idx_to_token.get(i, self.unk_token) for i in ids)

    def __len__(self):
        return len(self.vocab)


# dataset.py (Included for self-containment)
data = [
    ("hello", "bonjour"),
    ("how are you", "comment vas-tu"),
    ("i am fine", "je vais bien"),
    ("good morning", "bonjour"),
    ("thank you", "merci"),
    ("what is your name", "comment tu t'appelles"),
    ("see you later", "à plus tard"),
    ("i love to code", "j'adore coder"),
    ("this is a test", "c'est un test"),
    ("machine learning is fun", "l'apprentissage automatique est amusant"),
]

# On duplique pour avoir ~1000 exemples
import random
random.seed(42)
extended_data = data * 100
random.shuffle(extended_data)


class TranslationDataset(Dataset):
    def __init__(self, data, src_tokenizer, tgt_tokenizer, max_len=20):
        self.data = data
        self.src_tokenizer = src_tokenizer
        self.tgt_tokenizer = tgt_tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        src, tgt = self.data[idx]
        src_ids = self.src_tokenizer.encode(src)[:self.max_len]
        tgt_ids = self.tgt_tokenizer.encode(tgt)[:self.max_len]

        # Padding
        src_ids = src_ids + [self.src_tokenizer.token_to_idx["<pad>"]] * (self.max_len - len(src_ids))
        tgt_ids = tgt_ids + [self.tgt_tokenizer.token_to_idx["<pad>"]] * (self.max_len - len(tgt_ids))

        return torch.tensor(src_ids), torch.tensor(tgt_ids[:-1]), torch.tensor(tgt_ids[1:])  # input, target_in, target_out

# model.py (Included for self-containment)
import math

class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model=64, nhead=4, num_layers=2, dim_feedforward=128, max_len=50):
        super().__init__()
        self.src_embedding = nn.Embedding(src_vocab_size, d_model)
        self.tgt_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.pos_embedding = self.create_positional_encoding(max_len, d_model)

        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=nhead,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dim_feedforward=dim_feedforward,
            dropout=0.1,
            batch_first=True
        )
        self.fc_out = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(0.1)

    def create_positional_encoding(self, max_len, d_model):
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        return pe.unsqueeze(0)  # (1, max_len, d_model)

    def forward(self, src, tgt):
        src = self.src_embedding(src) + self.pos_embedding[:, :src.size(1), :]
        tgt = self.tgt_embedding(tgt) + self.pos_embedding[:, :tgt.size(1), :]

        src = self.dropout(src)
        tgt = self.dropout(tgt)

        output = self.transformer(src, tgt)
        return self.fc_out(output)


# Tokenizers
src_tokenizer = SimpleTokenizer()
tgt_tokenizer = SimpleTokenizer()

src_sentences = [src for src, tgt in extended_data]
tgt_sentences = [tgt for src, tgt in extended_data]

src_tokenizer.fit(src_sentences)
tgt_tokenizer.fit(tgt_sentences)

# Dataset & Dataloader
dataset = TranslationDataset(extended_data, src_tokenizer, tgt_tokenizer, max_len=15)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Model
model = Transformer(
    src_vocab_size=len(src_tokenizer),
    tgt_vocab_size=len(tgt_tokenizer),
    d_model=64,
    nhead=4,
    num_layers=2
)

criterion = nn.CrossEntropyLoss(ignore_index=src_tokenizer.token_to_idx["<pad>"])
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Training loop
model.train()
for epoch in range(20):
    total_loss = 0
    for src, tgt_in, tgt_out in tqdm(dataloader, desc=f"Epoch {epoch+1}/20"):
        optimizer.zero_grad()
        output = model(src, tgt_in)
        loss = criterion(output.view(-1, len(tgt_tokenizer)), tgt_out.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader):.4f}")

Epoch 1/20: 100%|██████████| 63/63 [00:02<00:00, 25.48it/s]


Epoch 1, Loss: 1.8610


Epoch 2/20: 100%|██████████| 63/63 [00:01<00:00, 33.26it/s]


Epoch 2, Loss: 0.3897


Epoch 3/20: 100%|██████████| 63/63 [00:01<00:00, 32.27it/s]


...

Epoch 18, Loss: 0.0042


Epoch 19/20: 100%|██████████| 63/63 [00:02<00:00, 23.29it/s]


Epoch 19, Loss: 0.0037


Epoch 20/20: 100%|██████████| 63/63 [00:01<00:00, 31.66it/s]

Epoch 20, Loss: 0.0034


In [18]:
# inference.py
def translate(model, sentence, src_tokenizer, tgt_tokenizer, max_len=15):
    model.eval()
    src_ids = torch.tensor([src_tokenizer.encode(sentence, add_special=True)]).to(next(model.parameters()).device)

    tgt_ids = [tgt_tokenizer.token_to_idx["<sos>"]]
    for _ in range(max_len):
        tgt_tensor = torch.tensor([tgt_ids])
        with torch.no_grad():
            output = model(src_ids, tgt_tensor)
        next_token = output[0, -1].argmax().item()
        tgt_ids.append(next_token)
        if next_token == tgt_tokenizer.token_to_idx["<eos>"]:
            break

    return tgt_tokenizer.decode(tgt_ids)

# Test
print(translate(model, "Bonjour", src_tokenizer, tgt_tokenizer))
print(translate(model, "Merci", src_tokenizer, tgt_tokenizer))

bonjour
bonjour


In [23]:
!git remote add origin https://github.com/oumarimat/Transformers